In [6]:
import pandas as pd
import numpy as np

#Open the src\data\gadm41_COD_2.json
import json
import os

from keras import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import SGD
from keras.utils import to_categorical


In [159]:

# Get cwd
cwd = os.getcwd()
# cd .. to get to the parent directory
cwd = os.path.dirname(cwd)

In [301]:
# load the csv src\data\LifeExp\API_SP.DYN.LE00.IN_DS2_en_csv_v2_4770434.csv
df = pd.read_csv(os.path.join(cwd, 'data', 'LifeExp', 'API_SP.DYN.LE00.IN_DS2_en_csv_v2_4770434.csv'))
# Get the rows with Indicator Name = Life expectancy at birth, total (years)
df = df[df['Indicator Name'] == 'Life expectancy at birth, total (years)']
# Exclude the rows which are county aggregates
#df = df[df['Country Code'] != 'COD']
df.drop(['Country Name', 'Indicator Name', 'Indicator Code', 'Unnamed: 66', '2021'], axis=1, inplace=True)

# load the csv src\data\LifeExp\API_SP.DYN.LE00.IN_DS2_en_csv_v2_4770434.csv
c_codes = pd.read_csv(os.path.join(cwd, 'data', 'country_codes.csv'))
# Icodes = c_codes['alpha-3'].values
iso_codes = c_codes['alpha-3'].values

#Keep the rows with the iso codes in the iso_codes list
df = df[df['Country Code'].isin(iso_codes)]
df.set_index('Country Code', inplace=True)

# keep only data from 1998 to 2018
df = df.loc[:, '1998':'2018']

#Turn into a nested dictionary
df = df.to_dict(orient='index')
# Read into a dataframe
df = pd.DataFrame.from_dict(df, orient='index')

# Turn into a nested dataframe
df = df.stack().reset_index()
# Rename the columns to 'country' and 'year' and 'life_exp'
df.columns = ['country', 'year', 'life_exp']

nested_df_lex = df.groupby(['country', 'year']).last()

In [302]:
import os
population = pd.read_excel(os.path.join(cwd, 'data', 'pop_bancomundial_excel.xlsx'))
population = population[population['Indicator Name'] == 'Población, total']

# Set the column names to string
population.columns = population.columns.astype(str)

population.drop(['Country Name', 'Indicator Name', 'Indicator Code', '2021'], axis=1, inplace=True)

# load the csv src\data\LifeExp\API_SP.DYN.LE00.IN_DS2_en_csv_v2_4770434.csv
c_codes = pd.read_csv(os.path.join(cwd, 'data', 'country_codes.csv'))
# Icodes = c_codes['alpha-3'].values
iso_codes = c_codes['alpha-3'].values

population = population[population['Country Code'].isin(iso_codes)]
population.set_index('Country Code', inplace=True)

# keep only data from 1998 to 2018
population = population.loc[:, '1998':'2018']

#Turn into a nested dictionary
population = population.to_dict(orient='index')
# Read into a dataframe
population = pd.DataFrame.from_dict(population, orient='index')

# Turn into a nested dataframe
population = population.stack().reset_index()
# Rename the columns to 'country' and 'year' and 'life_exp'
population.columns = ['country', 'year', 'population']

nested_df_pop = population.groupby(['country', 'year']).last()

In [307]:
# LOAD THE csv src\data\aqli_global_data.csv
df = pd.read_csv(os.path.join(cwd, 'data', 'aqli_global_data.csv'))
# keep columns iso_alpha3, and columsn qith pm in the name
df = df[['iso_alpha3'] + [col for col in df.columns if 'pm' in col]]
# Remove the substring 'pm' from the column names
df.columns = [col.replace('pm_', '') for col in df.columns]
df = df[df['iso_alpha3'].isin(iso_codes)]
df.pivot_table(index='iso_alpha3', aggfunc='mean')

#Turn into a nested dictionary
df = df.to_dict(orient='index')
# Read into a dataframe
df = pd.DataFrame.from_dict(df, orient='index')

df.set_index('iso_alpha3', inplace=True)

#add "pm_" to the column names
#df.columns = ['pm_' + col for col in df.columns]

# Turn into a nested dataframe
df = df.stack().reset_index()
# Rename the columns to 'country' and 'year' and 'life_exp'
df.columns = ['country', 'year', 'pm']

nested_df_pm = df.groupby(['country', 'year']).last()

In [309]:
#merge nested_df_lex and nested_df_pop
nested_df = pd.merge(nested_df_lex, nested_df_pop, on=['country', 'year'], how='inner')
nested_df = pd.merge(nested_df, nested_df_pm, on=['country', 'year'], how='inner')

In [328]:
# Save the nested_df to a csv
# drop countries in country_fake
nested_df = nested_df[~nested_df['country'].isin(county_fake)]
nested_df.to_csv(os.path.join(cwd, 'data', 'nested_df.csv'), index=False)

In [330]:
# Create the folder if it does not exist
if not os.path.exists(os.path.join(cwd, 'data', 'images')):
    os.makedirs(os.path.join(cwd, 'data', 'images'))

min_shape = 20
county_fake = []

all_df = pd.DataFrame()
#subset nested_df into dataframes for each country
for country in nested_df['country'].unique():
    nested_df_country = nested_df[nested_df['country'] == country]
    if len(nested_df_country) < 15:
        county_fake.append(country)
        min_shape = len(nested_df_country)

for country in nested_df['country'].unique():
    nested_df_country = nested_df[nested_df['country'] == country]
    #drop the column country
    if country in county_fake:
        continue
    else:
        nested_df_country.drop('country', axis=1, inplace=True)
        nested_df_country.tail(15).to_csv(os.path.join(cwd, 'data', 'images', str(country) + '.csv') , index=False)
        pd.concat([all_df, nested_df_country.tail(15)])

C:\Users\josep\AppData\Local\Temp\ipykernel_25252\2957448406.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nested_df_country.drop('country', axis=1, inplace=True)
C:\Users\josep\AppData\Local\Temp\ipykernel_25252\2957448406.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nested_df_country.drop('country', axis=1, inplace=True)
C:\Users\josep\AppData\Local\Temp\ipykernel_25252\2957448406.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

In [332]:
all_df

""


In [ ]:
# get the last 15 rows of each csv file
for country in nested_df['country'].unique():
    df = pd.read_csv(os.path.join(cwd, 'data', 'images', str(country) + '.csv'))
    df = df.tail(min_shape)
    df.to_csv(os.path.join(cwd, 'data', 'images', str(country) + '.csv') , index=False)

In [319]:
nested_df_country

,country,year,life_exp,population,pm
4116,ZWE,1998,46.82,11548364.00,16.35
4117,ZWE,1999,45.21,11716454.00,13.10
4118,ZWE,2000,44.69,11834676.00,13.22
4119,ZWE,2001,41.96,11910978.00,20.36
4120,ZWE,2002,44.56,11984644.00,15.98
4121,ZWE,2003,43.39,12075828.00,17.69
4122,ZWE,2004,44.50,12160881.00,15.33
4123,ZWE,2005,44.77,12224753.00,17.75
4124,ZWE,2006,45.36,12330490.00,15.93
4125,ZWE,2007,45.61,12450568.00,18.49


In [290]:
nested_df

,country,year,life_exp,population,pm
0,ABW,1998,73.49,84355.00,2.50
1,ABW,1999,73.56,86867.00,3.61
2,ABW,2000,73.57,89101.00,2.40
3,ABW,2001,73.65,90691.00,2.48
4,ABW,2002,73.73,91781.00,2.22
...,...,...,...,...,...
4132,ZWE,2014,58.85,13855753.00,14.38
4133,ZWE,2015,59.59,14154937.00,16.64
4134,ZWE,2016,60.31,14452704.00,14.90
4135,ZWE,2017,60.71,14751101.00,14.34


In [265]:
nested_df.to_csv(os.path.join(cwd, 'data', 'df.csv'), index=False)   

In [287]:
array_new = np.zeros([len(nested_df['country'].unique()), 21, 4])
for a, i in enumerate(nested_df['country'].unique()):
    array_new[a] = np.array(nested_df[nested_df['country'] == i][['year', 'life_exp', 'population', 'pm']])

ValueError: could not broadcast input array from shape (19,4) into shape (21,4)

,country,year,life_exp,population,pm
462,BMU,2000,77.89,61833.00,4.37
463,BMU,2001,77.89,62504.00,3.97
464,BMU,2002,78.09,62912.00,3.73
465,BMU,2003,78.33,63325.00,3.72
466,BMU,2004,78.49,63740.00,3.78
467,BMU,2005,78.69,64154.00,4.06
468,BMU,2006,78.89,64523.00,3.44
469,BMU,2007,78.93,64888.00,3.97
470,BMU,2008,79.24,65273.00,3.22
471,BMU,2009,79.39,65636.00,2.71


In [279]:
np.array(df[df['country'] == i].drop('country', axis=1))

array([['1998', 2.5],
       ['1999', 3.6099999],
       ['2000', 2.4000001],
       ['2001', 2.48],
       ['2002', 2.22],
       ['2003', 3.0899999],
       ['2004', 2.99],
       ['2005', 3.48],
       ['2006', 2.79],
       ['2007', 2.99],
       ['2008', 3.3900001],
       ['2009', 2.4200001],
       ['2010', 4.8200002],
       ['2011', 3.27],
       ['2012', 2.98],
       ['2013', 2.5899999],
       ['2014', 2.1600001],
       ['2015', 2.26],
       ['2016', 2.99],
       ['2017', 2.99],
       ['2018', 2.4000001]], dtype=object)

In [270]:
len(df['country'].unique())

236

In [200]:

    
nested_df.to_csv(os.path.join(cwd, 'RHINOS', 'data', 'fake_df.csv'), index=False)